In [1]:
import torch
import sys
import yaml
from torchvision import transforms, datasets
import torchvision
import numpy as np
import os
from sklearn import preprocessing
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from pathlib import Path
import numpy as np

In [2]:
sys.path.append('../')
from models.resnet_base_network import ResNet18

In [3]:
batch_size = 512
data_transforms = torchvision.transforms.Compose([transforms.ToTensor()])

In [4]:
config = yaml.load(open("../config/config.yaml", "r"), Loader=yaml.FullLoader)

In [5]:
class ImagesDataset(Dataset):
    def __init__(self, train, transform=None):
        """
        Args:
            train (boolean): train or test
        """
        self.paths = []
        self.labels = []
        for path in Path(f'../kneeKL224/{train}').glob('**/*'):
            folder, file = os.path.split(path)
            _, ext = os.path.splitext(path)
            if ext.lower() == '.png':
                self.paths.append(path)
                self.labels.append(folder[-1])
        self.transform = transform

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, index):
        path = self.paths[index]
        label = int(self.labels[index])
        img = Image.open(path)

        if self.transform:
            img = self.transform(img)

        return img, label

In [6]:
train_dataset = ImagesDataset(train = 'train', transform=data_transforms)

test_dataset = ImagesDataset(train = 'test', transform=data_transforms)

In [7]:
print("Input shape:", train_dataset[0][0].shape)

Input shape: torch.Size([1, 224, 224])


In [8]:
train_loader = DataLoader(train_dataset, batch_size=batch_size,
                          num_workers=0, drop_last=False, shuffle=True)

test_loader = DataLoader(test_dataset, batch_size=batch_size,
                          num_workers=0, drop_last=False, shuffle=True)

In [9]:
device = 'cpu' #'cuda' if torch.cuda.is_available() else 'cpu'
encoder = ResNet18(**config['network'])
output_feature_dim = encoder.projetion.net[0].in_features

In [10]:
#load pre-trained parameters
load_params = torch.load(os.path.join('../runs/Aug13_23-03-15_DESKTOP-SSCRTA9/checkpoints/model.pth'),
                         map_location=torch.device(torch.device(device)))

if 'online_network_state_dict' in load_params:
    encoder.load_state_dict(load_params['online_network_state_dict'])
    print("Parameters successfully loaded.")

# remove the projection head
encoder = torch.nn.Sequential(*list(encoder.children())[:-1])    
encoder = encoder.to(device)

Parameters successfully loaded.


In [11]:
encoder

Sequential(
  (0): Sequential(
    (0): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  

In [12]:
class LogisticRegression(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim)
        
    def forward(self, x):
        return self.linear(x)

In [13]:
logreg = LogisticRegression(output_feature_dim, 5)
logreg = logreg.to(device)

In [14]:
def get_features_from_encoder(encoder, loader):
    
    x_train = []
    y_train = []

    # get the features from the pre-trained model
    for i, (x, y) in enumerate(loader):
        with torch.no_grad():
            feature_vector = encoder(x)
            x_train.extend(feature_vector)
            y_train.extend(np.array(y))

            
    x_train = torch.stack(x_train)
    y_train = torch.tensor(y_train)
    return x_train, y_train

In [15]:
encoder.eval()
x_train, y_train = get_features_from_encoder(encoder, train_loader)
x_test, y_test = get_features_from_encoder(encoder, test_loader)

if len(x_train.shape) > 2:
    x_train = torch.mean(x_train, dim=[2, 3])
    x_test = torch.mean(x_test, dim=[2, 3])
    
print("Training data shape:", x_train.shape, y_train.shape)
print("Testing data shape:", x_test.shape, y_test.shape)

Training data shape: torch.Size([5778, 512]) torch.Size([5778])
Testing data shape: torch.Size([1656, 512]) torch.Size([1656])


In [16]:
def create_data_loaders_from_arrays(X_train, y_train, X_test, y_test):

    train = torch.utils.data.TensorDataset(X_train, y_train)
    train_loader = torch.utils.data.DataLoader(train, batch_size=64, shuffle=True)

    test = torch.utils.data.TensorDataset(X_test, y_test)
    test_loader = torch.utils.data.DataLoader(test, batch_size=512, shuffle=False)
    return train_loader, test_loader

In [17]:
scaler = preprocessing.StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train).astype(np.float32)
x_test = scaler.transform(x_test).astype(np.float32)

In [18]:
train_loader, test_loader = create_data_loaders_from_arrays(torch.from_numpy(x_train), y_train, torch.from_numpy(x_test), y_test)

In [19]:
optimizer = torch.optim.Adam(logreg.parameters(), lr=3e-4)
criterion = torch.nn.CrossEntropyLoss()
eval_every_n_epochs = 10

for epoch in range(200):
#     train_acc = []
    for x, y in train_loader:

        x = x.to(device)
        y = y.to(device)
        
        # zero the parameter gradients
        optimizer.zero_grad()        
        
        logits = logreg(x)
        predictions = torch.argmax(logits, dim=1)
        
        loss = criterion(logits, y)
        
        loss.backward()
        optimizer.step()
    
    total = 0
    if epoch % eval_every_n_epochs == 0:
        correct = 0
        for x, y in test_loader:
            x = x.to(device)
            y = y.to(device)

            logits = logreg(x)
            predictions = torch.argmax(logits, dim=1)
            
            total += y.size(0)
            correct += (predictions == y).sum().item()
            
        acc = 100 * correct / total
        print(f"Testing accuracy: {np.mean(acc)}")

Testing accuracy: 43.47826086956522
Testing accuracy: 54.04589371980676
Testing accuracy: 55.132850241545896
Testing accuracy: 54.95169082125604
Testing accuracy: 55.73671497584541
Testing accuracy: 55.85748792270532
Testing accuracy: 55.072463768115945
Testing accuracy: 54.95169082125604
Testing accuracy: 55.67632850241546
Testing accuracy: 55.49516908212561
Testing accuracy: 55.2536231884058
Testing accuracy: 55.79710144927536
Testing accuracy: 54.830917874396135
Testing accuracy: 54.64975845410628
Testing accuracy: 54.106280193236714
Testing accuracy: 54.64975845410628
Testing accuracy: 54.528985507246375
Testing accuracy: 54.589371980676326
Testing accuracy: 54.710144927536234
Testing accuracy: 54.408212560386474
